In [1]:
import numpy as np
import sympy
import tensorflow  as tf
from sympy import symbols
from sympy import simplify
from sympy.solvers import solve
from sympy.interactive.printing import init_printing
from sympy.matrices import Matrix, eye, zeros

# Matrix Multiplicaition

In [25]:
a = np.array([[0]*9])

In [26]:
a[0][0] = 1; a.transpose()

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [27]:
np.kron([[1], [0], [0]], [[1], [0], [0]]) == a.transpose()

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]])

In [82]:
a = np.random.normal(size=(3**9, 3**9)).astype('float32')
b = np.random.normal(size=(3**9, 1)).astype('float32')

expected = np.matmul(a, b)

In [84]:
def py_matmul(a, b):
    ra, ca = a.shape
    rb, cb = b.shape
    assert ca == rb, f"{ca} != {rb}"

    return np.matmul(a, b)

def py_tf(a, b):
    ra, ca = a.shape
    rb, cb = b.shape
    assert ca == rb, f"{ca} != {rb}"

    return tf.matmul(a, b)

def py_np(a, b):
    ra, ca = a.shape
    rb, cb = b.shape
    assert ca == rb, f"{ca} != {rb}"

    return a@b

def py_einsum(a, b):
    ra, ca = a.shape
    rb, cb = b.shape
    assert ca == rb, f"{ca} != {rb}"

    return np.einsum('ij,jk', a, b)
#assert result.shape == expected.shape
#assert np.allclose(result, expected, rtol=1e-02), (result, expected)

In [85]:
tf_a = tf.constant(a)
tf_b = tf.constant(b)
%time result = py_tf(tf_a, tf_b)

CPU times: total: 93.8 ms
Wall time: 124 ms


In [86]:
%time result = py_np(a, b)

CPU times: total: 359 ms
Wall time: 93.7 ms


In [87]:
%time result = py_matmul(a, b)

CPU times: total: 484 ms
Wall time: 98.7 ms


In [88]:
%time result = py_einsum(a, b)

CPU times: total: 172 ms
Wall time: 255 ms


In [117]:
a = np.random.random((60,60))
b = np.random.random((60,60))
ab = np.kron(a,b)
abe = np.einsum('ik,jl', a, b).reshape(3600,3600)

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [105]:
default_matrix = np.array([[0, 1, 0],
                          [1, 0, 0],
                          [0, 0, 1]], dtype=complex)

In [103]:
np.reshape(default_matrix, 1 * [2,2])

ValueError: cannot reshape array of size 9 into shape (2,2)

In [91]:
np.kron(default_matrix, np.eye(3))

array([[0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]])

# Decomposition

In [52]:
init_printing(use_unicode=False, wrap_line=False)
phi1 = symbols("varphi_1")
phi2 = symbols("varphi_2")
phi3 = symbols("varphi_3")
phi4 = symbols("varphi_4")
phi5 = symbols("varphi_5")
phi6 = symbols("varphi_6")
phix  = symbols("varphi_x")
x, x1, x2, y, y1, y2, z, z1, z2  = symbols("x x_1 x_2 y y_1 y_2 z z_1 z_2")
theta = symbols("theta")
theta1 = symbols("theta_1")
theta2 = symbols("theta_2")
theta3 = symbols("theta_3")
theta4 = symbols("theta_4")
a, b, c, d, e, f, g, h, i = symbols("a b c d e f g h i")

In [53]:
U_matrix = Matrix([[a, b, c],
                   [d, e, f],
                   [g, h, i]]); U_matrix

[a  b  c]
[       ]
[d  e  f]
[       ]
[g  h  i]

In [54]:
def UD(phi1, phi2, phi3):
    return Matrix([[sympy.exp(1j * phi1), 0, 0],
                    [0, sympy.exp(1j * phi2), 0],
                    [0, 0, sympy.exp(1j * phi3)]])
def Z01(phi):
    return Matrix([[sympy.exp(1j * phi), 0, 0],
                   [0, 1, 0],
                   [0, 0, 1]])
def Z12(phi):
    return Matrix([[1, 0, 0],
                   [0, 1, 0],
                   [0, 0, sympy.exp(1j * phi)]])
def R01(theta):
    return Matrix([[sympy.cos(theta / 2), -1j * sympy.sin(theta / 2), 0],
                   [-1j * sympy.sin(theta / 2), sympy.cos(theta / 2), 0],
                   [0, 0, 1]])
def R12(theta):
    return Matrix([[1, 0, 0],
                   [0, sympy.cos(theta / 2), -1j * sympy.sin(theta / 2)],
                   [0, -1j * sympy.sin(theta / 2), sympy.cos(theta / 2)]])
def RZ01(theta):
    return Matrix([[sympy.exp(-1j * theta / 2), 0, 0],
                   [0, sympy.exp(1j * theta / 2), 0],
                   [0, 0, 1]])
def RZ12(theta):
    return Matrix([[1, 0, 0],
                   [0, sympy.exp(-1j * theta / 2), 0],
                   [0, 0, sympy.exp(1j * theta / 2)]])

In [55]:
def G12(theta, phi):
    return Z12(phi)*R12(theta)*Z12(-phi)

In [56]:
def G01(theta, phi):
    return Z01(-phi)*R01(theta)*Z01(phi)

In [ ]:
U_D()

In [134]:
RZ01(phi2)

[ -0.5*I*varphi_2                    ]
[e                        0         0]
[                                    ]
[                   0.5*I*varphi_2   ]
[       0          e                0]
[                                    ]
[       0                 0         1]

In [57]:
A = simplify(UD(phi6,phi5,phi4)*G01(theta3,phi3)*G12(theta2,phi2)*G01(theta1,phi1)); A

[/   1.0*I*(varphi_1 + varphi_6)    /theta_1\    /theta_3\    /theta_2\    1.0*I*(varphi_3 + varphi_6)    /theta_1\    /theta_3\\  -1.0*I*varphi_3        /   1.0*I*(varphi_1 + varphi_6)    /theta_3\    /theta_1\    /theta_2\    1.0*I*(varphi_3 + varphi_6)    /theta_1\    /theta_3\\  -1.0*I*(varphi_1 + varphi_3)        -1.0*I*(varphi_2 + varphi_3 - varphi_6)    /theta_2\    /theta_3\]
[|- e                           *sin|-------|*sin|-------|*cos|-------| + e                           *cos|-------|*cos|-------||*e                 1.0*I*|- e                           *sin|-------|*cos|-------|*cos|-------| - e                           *sin|-------|*cos|-------||*e                              -1.0*e                                       *sin|-------|*sin|-------|]
[\                                  \   2   /    \   2   /    \   2   /                                   \   2   /    \   2   //                         \                                  \   2   /    \   2   /    \   2   /  

In [129]:
B = simplify(RZ01(x1)*RZ12(x2)*G01(theta3,phi3)*G12(theta2,phi2)*RZ01(y1)*RZ12(y2)*G01(theta1,phi1)*RZ01(z1)*RZ12(z2))

In [130]:
sol = sympy.solve(A-B, (x1, x2, y1, y2, z1, z2))

In [132]:
sol

In [ ]:
print(sol[0][0])

In [121]:
B = B.subs(x1, phi5-phi6)
B = B.subs(x2, 2*phi4+phi5+phi6)
B = B.subs(y1, 2*phi4)
B = B.subs(y2, 0)
B = B.subs(z2, 0)

In [122]:
simplify(B)

[     -I*varphi_4 - I*varphi_5    /theta_1\    /theta_3\    1.0*I*varphi_1 - I*varphi_3 + 1.0*I*varphi_6    /theta_1\    /theta_3\    /theta_2\            -I*varphi_3 + 1.0*I*varphi_6    /theta_3\    /theta_1\    /theta_2\      -I*varphi_1 - I*varphi_4 - I*varphi_5    /theta_1\    /theta_3\        -1.0*I*(varphi_2 + varphi_3 - varphi_6)    /theta_2\    /theta_3\]
[    e                        *cos|-------|*cos|-------| - e                                            *sin|-------|*sin|-------|*cos|-------|       - I*e                            *sin|-------|*cos|-------|*cos|-------| - I*e                                     *sin|-------|*cos|-------|  -1.0*e                                       *sin|-------|*sin|-------|]
[                                 \   2   /    \   2   /                                                    \   2   /    \   2   /    \   2   /                                            \   2   /    \   2   /    \   2   /                                              

In [92]:
simplify(A)

[/   1.0*I*(varphi_1 + varphi_6)    /theta_1\    /theta_3\    /theta_2\    1.0*I*(varphi_3 + varphi_6)    /theta_1\    /theta_3\\  -1.0*I*varphi_3        /   1.0*I*(varphi_1 + varphi_6)    /theta_3\    /theta_1\    /theta_2\    1.0*I*(varphi_3 + varphi_6)    /theta_1\    /theta_3\\  -1.0*I*(varphi_1 + varphi_3)        -1.0*I*(varphi_2 + varphi_3 - varphi_6)    /theta_2\    /theta_3\]
[|- e                           *sin|-------|*sin|-------|*cos|-------| + e                           *cos|-------|*cos|-------||*e                 1.0*I*|- e                           *sin|-------|*cos|-------|*cos|-------| - e                           *sin|-------|*cos|-------||*e                              -1.0*e                                       *sin|-------|*sin|-------|]
[\                                  \   2   /    \   2   /    \   2   /                                   \   2   /    \   2   //                         \                                  \   2   /    \   2   /    \   2   /  

In [13]:
CNOT = state00@np.transpose(state00) +\
state01@np.transpose(state01) +\
state02@np.transpose(state02) +\
state10@np.transpose(state12) +\
state11@np.transpose(state10) +\
state12@np.transpose(state11) +\
state20@np.transpose(state21) +\
state21@np.transpose(state22) +\
state22@np.transpose(state20)

In [14]:
CNOT

array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0]])

In [18]:
def U_D(phi_i, phi_j, phi_k):
    return np.array([[np.exp(1j*phi_i), 0, 0],
                     [0, np.exp(1j*phi_j), 0],
                     [0, 0, np.exp(1j*phi_k)]])

In [19]:
def rx01(parameter):
    return np.array([[np.cos(parameter / 2), -1j * np.sin(parameter / 2), 0],
                    [-1j * np.sin(parameter / 2), np.cos(parameter / 2), 0],
                    [0, 0, 1]])

In [20]:
def rx12(parameter):
    return np.array([[1, 0, 0],
                    [0, np.cos(parameter / 2), -1j * np.sin(parameter / 2)],
                    [0, -1j * np.sin(parameter / 2), np.cos(parameter / 2)]])

In [21]:
def P01(parameter):
    return np.array([[np.exp(1j*parameter), 0, 0],
                    [0, 1, 0],
                    [0, 0, 1]])

In [22]:
def P12(parameter):
    return np.array([[1, 0, 0],
                    [0, 1, 0],
                    [0, 0, np.exp(1j*parameter)]])

In [23]:
def R_01_offset(theta, phi):
    return P01(-phi)@rx01(theta)@P01(phi)

In [24]:
def R_12_offset(theta, phi):
    return P12(phi)@rx12(theta)@P12(-phi)

In [17]:
def get_parameter(U):
    if abs(np.absolute(U[2, 2]) - 1) < 1e-6:
        theta_1 = phi_1 = theta_2 = phi_2 = 0
        phi_4 = np.angle(U[2, 2])
        phi_5 = np.angle(U[1, 1])
        phi_6 = np.angle(U[0, 0])
        # phi_3 = phi_6 - pi/2 - np.angle(U[0, 1])
        phi_3 = np.angle(U[1, 0]) - phi_5 + pi/2
        theta_3 = 2*np.arccos(np.absolute(U[1, 1]))
    else:
        phi_4 = np.angle(U[2, 2])
        theta_2 = 2*np.arccos(np.round(np.absolute(U[2, 2]), 6))
        phi_2 = np.angle(U[2, 1]) - phi_4 + pi/2
        phi_1 = np.angle(-U[2, 0]) - phi_2 - phi_4
        theta_1 = 2*np.arccos(np.round(np.absolute(U[2, 1])/np.sin(theta_2/2), 6))
        theta_3 = 2*np.arccos(np.round(np.absolute(U[1, 2])/np.sin(theta_2/2), 6))
        phi_5 = np.angle(U[1, 2]) + phi_2 + pi/2
        phi_3 = np.angle(np.cos(theta_1/2) * np.cos(theta_2/2) * np.cos(theta_3/2) - U[1, 1]*np.exp(-1j*phi_5)) + phi_1
        phi_6 = np.angle(-U[0, 2]) + phi_3 + phi_2
    return theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6

def reconstruct(theta_1, theta_2, theta_3, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6):
    return U_d(phi_6, phi_5, phi_4) @ R01(phi_3, theta_3) @ R12(phi_2, theta_2) @ R01(phi_1, theta_1)